# Vegeval potato detection model comparison
Richard Aljaste, 2023 University of Tartu

## Library and model imports

In [ ]:
!git clone https://github.com/ultralytics/yolov5 # YOLOv5
!git clone https://github.com/meituan/YOLOv6 # YOLOv6
!git clone https://github.com/wongkinyiu/yolov7 # YOLOv7 prod
#!pip install ultralytics # YOLOv8 prod
!git clone --branch fix/tflite_integer_quant https://github.com/motokimura/ultralytics # YOLOv8 export fix

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler	

import torch, os, random, time, shutil, sys
from google.colab import drive
import tensorflow as tf
import numpy as np

## Data preparation

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/dataset.zip" -d "/content/dataset/"

In [4]:
# Split data into train, validation sets by 80% and 20% 
image_sets = {'train': [], 'valid': []}
all_image_names = os.listdir(f"/content/dataset/images/")
random.shuffle(all_image_names)
nr_of_images = len(all_image_names)

for j, image_name in enumerate(all_image_names):
  percentage = j / nr_of_images * 100
  if percentage <= 80:
    image_sets['train'].append(image_name)
  else:
    image_sets['valid'].append(image_name)

for image_set in image_sets.keys():
  if not os.path.isdir(f"/content/dataset/{image_set}"):
    os.mkdir(f"/content/dataset/{image_set}")
    os.mkdir(f"/content/dataset/{image_set}/images")
    os.mkdir(f"/content/dataset/{image_set}/labels")

  for image in image_sets.get(image_set):
    file_name = image[:-4]
    os.rename(f"/content/dataset/images/{image}", f"/content/dataset/{image_set}/images/{image}")
    os.rename(f"/content/dataset/labels/{file_name}.txt", f"/content/dataset/{image_set}/labels/{file_name}.txt")

os.removedirs(f"/content/dataset/images")
os.removedirs(f"/content/dataset/labels")

In [5]:
# Read classes
classes_file = open("/content/dataset/classes.txt", "r")
classes = []
for c in classes_file.readlines():
  classes.append(c.replace('\n', ''))
classes_file.close()

In [6]:
# Create required .yaml file describing our dataset in YOLO format
data_file = open("/content/dataset/data.yaml", "w")
data_file.write("""train: /content/dataset/train/
val: /content/dataset/valid/
nc: """ + str(len(classes)) + 
"""
names: """ + str(classes))
data_file.close()

## Model training, evaluation

### YOLOv5

In [ ]:
%cd /content/yolov5
%pip install -r requirements.txt

In [ ]:
# YOLOv5n
!python train.py --data '/content/dataset/data.yaml' --epochs 250 --weights 'yolov5n.pt' --batch-size 32 --workers 8 --cache ram --name 'yolov5n' --imgsz 640

In [ ]:
# Evaluate
!python val.py --weights '/content/yolov5/runs/train/yolov5n/weights/best.pt' --data '/content/dataset/data.yaml' --name 'yolov5n-test' --imgsz 640

In [ ]:
# YOLOv5n video test
!python detect.py --weights '/content/yolov5/runs/train/yolov5n/weights/best.pt' --source '/content/dataset/vegeval_demo.mp4' --name 'yolov5n-labelled-video' --conf-thres 0.25

In [ ]:
# Export to tflite and compile for edgetpu
!python export.py --weights '/content/yolov5/runs/train/yolov5n/weights/best.pt' --include edgetpu

In [ ]:
# Evaluate int8
!python val.py --weights '/content/yolov5/runs/train/yolov5n/weights/best-int8.tflite' --data '/content/dataset/data.yaml' --name 'yolov5n-test-int8'

In [ ]:
shutil.copytree('/content/yolov5/runs/train', '/content/runs/train', dirs_exist_ok=True)

### YOLOv6

In [ ]:
%cd /content/YOLOv6
%pip install -r requirements.txt

In [ ]:
# Create custom dataset directory structure
os.mkdir('/content/dataset-v6')
os.mkdir('/content/dataset-v6/images')
os.mkdir('/content/dataset-v6/labels')
shutil.copytree('/content/dataset/train/images', '/content/dataset-v6/images/train')
shutil.copytree('/content/dataset/valid/images', '/content/dataset-v6/images/valid')
shutil.copytree('/content/dataset/train/labels', '/content/dataset-v6/labels/train')
shutil.copytree('/content/dataset/valid/labels', '/content/dataset-v6/labels/valid')

In [ ]:
# Create custom .yaml file
data_file = open("/content/dataset-v6/data.yaml", "w")
data_file.write("""train: /content/dataset-v6/images/train
val: /content/dataset-v6/images/valid
nc: """ + str(len(classes)) + 
"""
names: """ + str(classes))
data_file.close()

In [ ]:
# YOLOv6-N
!python tools/train.py --batch 32 --epochs 250 --conf "configs/yolov6n_finetune.py" --data "/content/dataset-v6/data.yaml" --fuse_ab --device 0 --name "yolov6n"

In [ ]:
# Evaluate
!python tools/eval.py --data "/content/dataset/data.yaml" --batch 32 --weights "runs/train/yolov6n3/weights/best_ckpt.pt" --task val --name "yolov6n-test"

In [ ]:
# Inference
!python tools/infer.py --weights "runs/train/yolov6n3/weights/best_ckpt.pt" --source "/content/dataset/vegeval_demo.mp4" --name "yolov6n-labelled-video"

In [ ]:
# Export
!pip install openvino-dev
!pip install openvino2tensorflow
!pip install onnx onnxsim
import onnx
import onnxsim

#!python deploy/OpenVINO/export_openvino.py --weights "runs/train/yolov6n3/weights/best_ckpt.pt" --img 640 --batch 1
!mo --input_model 'runs/train/yolov6n3/weights/best_ckpt.onnx'
!openvino2tensorflow --model_path 'best_ckpt.xml' --output_edgetpu

In [ ]:
!python tools/eval.py --data "/content/dataset-v6/data.yaml" --batch 32 --weights "runs/train/yolov6n3/weights/best_ckpt.pt" --task val --name "yolov6n-test"

In [ ]:
shutil.copytree('/content/YOLOv6/runs/train', '/content/runs/train', dirs_exist_ok=True)

'/content/runs/train'

### YOLOv7

In [ ]:
os.remove("/content/dataset/train/labels.cache")
os.remove("/content/dataset/valid/labels.cache")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8" # required to fix locale bug

%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt
%pip install -qr requirements.txt

In [ ]:
# YOLOv7 tiny 640
!python train.py --workers 8 --batch-size 32 --data "/content/dataset/data.yaml" --img 640 --cfg "/content/yolov7/cfg/training/yolov7-tiny.yaml" --weights "/content/yolov7/yolov7-tiny.pt" --name "yolov7-tiny" --hyp "/content/yolov7/data/hyp.scratch.tiny.yaml" --epochs 250 --cache-images

In [ ]:
# Evaluate
!python test.py --data "/content/dataset/data.yaml" --img 640 --batch 32 --device 0 --weights "runs/train/yolov7-tiny/weights/best.pt" --name "yolov7tiny-test"

In [ ]:
# YOLOv7 tiny video test
%cd /content/yolov7
!python detect.py --weights "runs/train/yolov7-tiny/weights/best.pt" --conf 0.25 --img-size 640 --source "/content/dataset/vegeval_demo.mp4" --name "yolov7tiny-labelled-video"

In [ ]:
# Export
%cd /content/yolov7
!pip install -r requirements.txt
!pip install openvino-dev
!pip install openvino2tensorflow
!pip install onnx onnxsim
import onnx
import onnxsim

!python export.py --weights '/content/yolov7/runs/train/yolov7-tiny/weights/best.pt' --simplify
!mo --input_model '/content/yolov7/runs/train/yolov7-tiny/weights/best.onnx'
!openvino2tensorflow --model_path '/content/yolov7/best.xml' --output_edgetpu

In [ ]:
shutil.copytree('/content/yolov7/runs/train', '/content/runs/train', dirs_exist_ok=True)

### YOLOv8

In [ ]:
# Delete label cache of previous models
os.remove("/content/dataset/train/labels.cache")
os.remove("/content/dataset/valid/labels.cache")

In [ ]:
# YOLOv8n
%cd /content/ultralytics
!pip install .
from ultralytics import YOLO
yolov8n = YOLO("yolov8n.pt")
yolov8n.train(data="/content/dataset/data.yaml", epochs=250, name="yolov8n", device=0, batch=32, workers=8, imgsz=640, cache=True)

In [ ]:
# Evaluate
yolov8n.val()

In [ ]:
# Inference
yolov8n(model="/content/yolov8/runs/detect/yolov8n/weights/best.pt", source="/content/dataset/vegeval_demo.mp4", name="yolov8n-labelled-video", save=True)

In [ ]:
# Export to tflite and compile for edgetpu
#yolov8n.export(format="edgetpu")

In [ ]:
# Export using int8 quantization fix
yolov8n = YOLO("/content/ultralytics/runs/detect/yolov8n/weights/best.pt")
yolov8n.export(format='edgetpu')

In [ ]:
yolov8n = YOLO("/content/ultralytics/runs/detect/yolov8n/weights/best_saved_model/best_full_integer_quant.tflite")
yolov8n.val(data='/content/dataset/data.yaml')

In [ ]:
shutil.copytree('/content/ultralytics/runs/detect', '/content/runs/train', dirs_exist_ok=True)

### EfficientDet-Lite2

In [ ]:
os.mkdir('/content/efficientdet-lite2')
%cd /content/efficientdet-lite2

In [ ]:
# Install and import required packages (incompatible with above code, comment out any breaking imports)
!pip install -q tflite-model-maker
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

In [ ]:
# AutoML export format is not supported by Label Studio so we need convert to it from COCO format
def convert_coco_json_to_csv(filename, image_sets, labels):
    import json, random
    
    s = json.load(open(filename, 'r'))
    
    # Remember image paths by id
    images = {}
    for im in s['images']:
        images[im['id']] = {
            'path': im['file_name'].split('/')[-1], # Split required due to Label Studio bug
            'width': im['width'],
            'height': im['height']
        }
    train_set = image_sets.get('train')
    validation_set = image_sets.get('valid')

    nr_of_annotations = len(s['annotations']) - 1

    # Write to Google Cloud AutoML format .csv
    out_file = filename[:-5] + '.csv'
    out = open(out_file, 'w')

    # set,path,label,x_min,y_min,,,x_max,y_max,,
    out.write('set,path,label,x_min,y_min,,,x_max,y_max,,\n')

    for ann in s['annotations']:
        width = images[ann['image_id']]['width']
        height = images[ann['image_id']]['height']

        x_min = ann['bbox'][0] / width
        x_max = (ann['bbox'][0] + ann['bbox'][2]) / width
        y_min = ann['bbox'][1] / height
        y_max = (ann['bbox'][1] + ann['bbox'][3]) / height

        # Split images into train, validation and test sets
        path = images[ann['image_id']]['path']
        if path in train_set:
            img_set = 'TRAIN'
            path = 'train/images/' + path
        elif path in validation_set:
            img_set = 'VALIDATION'
            path = 'valid/images/' + path
        else:
          sys.exit(path + " not found in any provided image sets!")

        label = labels[int(ann['category_id'])]
        out.write('{},{},{},{},{},{},{},{},{},{},{}\n'.format(img_set, path, label, x_min, y_min, '', '', x_max, y_max, '', ''))
    out.close()

In [ ]:
# Create required .csv file describing our dataset in AutoML format
convert_coco_json_to_csv("/content/dataset/result.json", image_sets, ['StandardQ', 'LowQ', 'MediumQ'])

In [ ]:
spec = model_spec.get('efficientdet_lite2', tflite_max_detections=100)
spec.config.num_classes = len(classes)
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('/content/dataset/result.csv', '/content/dataset')

In [ ]:
# EfficientDet-Lite2
model = object_detector.create(train_data, model_spec=spec, batch_size=32, epochs=250, validation_data=validation_data)

In [ ]:
#Evaluate
model.evaluate(validation_data)

In [ ]:
# Export
model.export(export_dir='/content/efficientdet-lite2', tflite_filename='efficientdet-lite2.tflite')

In [ ]:
model = object_detector

In [ ]:
# Evaluate INT8
model.evaluate_tflite('efficientdet-lite2.tflite', validation_data)

In [ ]:
%cd /content/efficientdet-lite2
!edgetpu_compiler '/content/efficientdet-lite2/runs/train/efficientdet-lite2.tflite'

In [ ]:
shutil.copytree('/content/efficientdet-lite2', '/content/runs/train', dirs_exist_ok=True)

## Analysis

In [ ]:
# Copy 
shutil.copytree('/content/runs', '/content/drive/MyDrive/Vegeval/models', dirs_exist_ok=True)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/runs"